In [1]:
!pip install ibm_watson

In [2]:
!pip install PyJWT==1.7.1

<p>
<ur>
<li>API ID :dchaF_ipcbVrS4icXbMYZK7VHqIbzDKQWEc0CVrIEk7e </li>
<li>URL:https://api.jp-tok.personality-insights.watson.cloud.ibm.com/instances/9282a647-3e23-4ff0-838a-2c7842d5cbf7</li>
</p>


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1>IBM Personality Insight</h1>
<p>レビューデータをIBM Personality Insightに渡し、Big5に数量化した後、各施設のBig5値をcsvに保存する。</p>

In [4]:
#必要なもージュールのインポート
import json # json解析モジュールのインポート
from ibm_watson import PersonalityInsightsV3 # PersonalityInsights
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from os.path import join, dirname 
import csv

In [5]:
import glob

files = glob.glob("/content/drive/My Drive/output_data/*.txt")
len(files)

1000

In [6]:
def analyze_personality():
  #------------------------------入力----------------------------------
  authenticator = IAMAuthenticator('pRNOl10olOiQSOoYwcB7YPWM4mqtkSwiyB_7x1iqq5kc') #APIkeyを入力
  #---------------------------------------------------------------------

  service = PersonalityInsightsV3(
      version='2018-10-30',
      authenticator= authenticator)
  #------------------------------入力----------------------------------
  service.set_service_url('https://api.jp-tok.personality-insights.watson.cloud.ibm.com/instances/a5d374a7-e87f-4c6e-81db-55d5e928fcb8') #URLを入力
  #--------------------------------------------------------------------
  # 性格を分析
  files = glob.glob("/content/drive/My Drive/output_data/*.txt")
  for i,file in enumerate(files):
    if (i + 1) % 50 == 0 :print(str(i+1)+'times')
    with open(file, 'r') as profile_text:  #解析したいtxtファイルを絶対パスで指定
        profile = service.profile(
            profile_text.read(),
            'application/json',
            content_language='en',
            accept_language='ja').get_result()

        # ファイルに書き込み
        with open('/content/drive/My Drive/result_data/result'+str(i+1)+'.json', 'w') as f:
                json.dump(profile, f, ensure_ascii=False, indent=2)

In [7]:
analyze_personality() #関数を実行

50times
100times
150times
200times
250times
300times
350times
400times
450times
500times
550times
600times
650times
700times
750times
800times
850times
900times
950times
1000times


<h1>JSONファイルをCSVに変更する。</h1>

In [8]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 13 14:23:53 2020

JSONからCSV出力を行う
@author: Souichirou Kikuchi
"""
import argparse
import json
import csv

print('--- start program ---')
files = glob.glob("/content/drive/My Drive/result_data/*.json")
datas = [] 

for i, file in enumerate(files):
    # パラメータの設定
    parser = argparse.ArgumentParser(
            formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument(
            '--injson', help=file, required=True)
    parser.add_argument(
            '--otcsv', help='/content/drive/My Drive/result_data/result.csv', required=True)
    args = parser.parse_args(args=['--injson',file, '--otcsv', '/content/drive/My Drive/result_data/result.csv'])

    # JSONファイルの読み込み
    with open(args.injson, 'r', encoding='utf_8') as f:
        if i + 1 % 10 == 0:
            print(str(i+1)+'times')
        json_data = json.load(f) 
        word_count = json_data['word_count']
        big5_openness = json_data["personality"][0]['percentile']
        big5_conscientiousness = json_data["personality"][1]['percentile']
        big5_extraversion = json_data["personality"][2]['percentile']
        big5_agreeableness = json_data["personality"][3]['percentile']
        big5_neuroticism= json_data["personality"][4]['percentile']                
        datas.append([i+1,word_count, big5_openness, big5_conscientiousness,big5_extraversion,big5_agreeableness,big5_neuroticism])

#CSVに出力
with open(args.otcsv, 'w', newline='') as csvFile: 
    print('---started writing csv---')
    out_csv = csv.writer(csvFile, delimiter=',',quotechar='"', quoting=csv.QUOTE_NONNUMERIC) 
    out_csv.writerow(['times','word_count','big5_openness', 'big5_conscientiousness','big5_extraversion','big5_agreeableness','big5_neuroticism'])
    for data in datas:
        out_csv.writerow(data)
    print('---finished writing csv---')

--- start program ---
---started writing csv---
---finished writing csv---
